# NYCT Delay Alerts: Data Preparation

In this notebook, we will load a snapshot of the NYCT alerts dataset and perform simple natural language processing (NLP) in order to extract the issue causing the delay and the train stop at (or near) which the issue occurred.

After setting up a virtual environment (i.e. `python3 -m venv venv`), you can installed the necessary dependencies with `pip install -r requirements.txt`.

# Import dependencies

In [214]:
import pandas as pd
import numpy as np
import re
from collections import namedtuple, Counter
from datetime import datetime
from enum import StrEnum, auto
import json

In [215]:
# Additional setup
pd.options.mode.chained_assignment = None

# Read data

## Alerts data

Here we load the MTA alerts dataset, which is available for download [here](https://data.ny.gov/Transportation/MTA-Service-Alerts-Beginning-April-2020/7kct-peq7). At the time of writing, the dataset contains alerts from April 2020 to August 2024.

In [216]:
alerts_df = pd.read_csv('../../data/raw-data/service-alerts.csv')

/var/folders/rc/_s472dfx74l_g28snwc02ktr0000gn/T/ipykernel_52281/2574609879.py:1: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  alerts_df = pd.read_csv('../../data/raw-data/service-alerts.csv')


In [217]:
alerts_df.head(5)

,Alert ID,Event ID,Update Number,Date,Agency,Status Label,Affected,Header,Description
0,283131,135287,0,01/08/2024 07:17:00 PM,LIRR,extra-service,Babylon Branch,tweet,g
1,253262,119758,0,09/15/2023 01:05:00 PM,NYCT Subway,boarding-change,6,NaN,NaN
2,198453,92346,0,02/04/2023 05:50:00 AM,LIRR,extra-service,City Terminal Zone,tweet,NaN
3,179799,82495,0,11/04/2022 03:18:00 AM,LIRR,station-notice,City Terminal Zone,.,NaN
4,158731,70726,0,07/30/2022 04:01:00 AM,LIRR,service-change,Port Jefferson Branch,Weekend Changes,NaN


In [218]:
def filter_by_status_label(df, status_label):
    ''' Filter df by whether its status labels contains status_label '''
    return df[df["Status Label"].apply(lambda x: status_label in x.split(" | "))]

In [219]:
alerts_nyct = alerts_df[alerts_df["Agency"] == "NYCT Subway"]
nyct = filter_by_status_label(alerts_nyct, "delays")

In [220]:
# Minor cleaning
nyct["Header"] = nyct["Header"].replace(np.nan, None)
nyct["Description"] = nyct["Description"].replace(np.nan, None)

In [221]:
nyct.sample(20)

,Alert ID,Event ID,Update Number,Date,Agency,Status Label,Affected,Header,Description
308901,34049,7671,0,10/02/2020 10:45:00 AM,NYCT Subway,delays,Q,Northbound Q trains are delayed while we remov...,None
319181,23635,5009,0,08/13/2020 11:39:00 AM,NYCT Subway,delays,4,Northbound 4 trains are holding in stations wh...,None
300657,41721,11079,0,11/24/2020 04:14:00 AM,NYCT Subway,delays,2 | 3,Southbound 2 and 3 trains are delayed while we...,None
167600,176534,80683,2,10/19/2022 09:36:00 PM,NYCT Subway,delays,1 | 2 | 3,1 2 3 trains are running with delays in both d...,None
338069,5049,1110,3,05/19/2020 11:41:00 AM,NYCT Subway,delays,A,Expect longer waits for northbound A trains in...,None
69781,275245,131310,2,12/10/2023 07:17:00 AM,NYCT Subway,delays,1 | 2,Northbound 1 2 trains are running with delays ...,Northbound 1 2 trains have resumed making loca...
213179,130883,54314,1,03/17/2022 02:33:00 PM,NYCT Subway,delays,4 | 5,Southbound 4 5 trains are running with delays ...,Southbound 4 5 express service has resumed fro...
183333,160805,71909,2,08/08/2022 10:25:00 PM,NYCT Subway,delays,R,Northbound R trains are running with delays af...,Northbound R trains have resumed making regula...
69739,275287,131337,1,12/10/2023 03:18:00 PM,NYCT Subway,delays,4 | 5,4 trains are delayed in both directions while ...,The last stop on southbound 4 trains is Bowlin...
79638,265330,126043,1,10/31/2023 10:28:00 PM,NYCT Subway,delays,G,Church Av-bound G trains are running with dela...,None


## Station data

We'll also find it useful later to have data on the NYCT stations. This data is available for download [here](https://data.ny.gov/Transportation/MTA-Subway-Stations/39hk-dx4f/about_data).

In [222]:
stations_df = pd.read_csv('../../data/raw-data/subway-stations.csv')

In [223]:
stations_df

,GTFS Stop ID,Station ID,Complex ID,Division,Line,Stop Name,Borough,CBD,Daytime Routes,Structure,GTFS Latitude,GTFS Longitude,North Direction Label,South Direction Label,ADA,ADA Northbound,ADA Southbound,ADA Notes,Georeference
0,R01,1,1,BMT,Astoria,Astoria-Ditmars Blvd,Q,False,N W,Elevated,40.775036,-73.912034,Last Stop,Manhattan,0,0,0,NaN,POINT (-73.912034 40.775036)
1,R03,2,2,BMT,Astoria,Astoria Blvd,Q,False,N W,Elevated,40.770258,-73.917843,Astoria,Manhattan,1,1,1,NaN,POINT (-73.917843 40.770258)
2,R04,3,3,BMT,Astoria,30 Av,Q,False,N W,Elevated,40.766779,-73.921479,Astoria,Manhattan,0,0,0,NaN,POINT (-73.921479 40.766779)
3,R05,4,4,BMT,Astoria,Broadway,Q,False,N W,Elevated,40.761820,-73.925508,Astoria,Manhattan,0,0,0,NaN,POINT (-73.925508 40.76182)
4,R06,5,5,BMT,Astoria,36 Av,Q,False,N W,Elevated,40.756804,-73.929575,Astoria,Manhattan,0,0,0,NaN,POINT (-73.929575 40.756804)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
491,S15,517,517,SIR,Staten Island,Prince's Bay,SI,False,SIR,Open Cut,40.525507,-74.200064,Ferry,South Shore,0,0,0,NaN,POINT (-74.200064 40.525507)
492,S14,518,518,SIR,Staten Island,Pleasant Plains,SI,False,SIR,Embankment,40.522410,-74.217847,Ferry,South Shore,0,0,0,NaN,POINT (-74.217847 40.52241)
493,S13,519,519,SIR,Staten Island,Richmond Valley,SI,False,SIR,Open Cut,40.519631,-74.229141,Ferry,Tottenville,0,0,0,NaN,POINT (-74.229141 40.519631)
494,S09,522,522,SIR,Staten Island,Tottenville,SI,False,SIR,At Grade,40.512764,-74.251961,Ferry,Last Stop,1,1,1,NaN,POINT (-74.251961 40.512764)


# Data processing

There are a number of improvements we may want to make in order to make this dataset more information-rich. Ultimately, we would like to identify common issues that cause delays, and identify which stations are particularly issue-prone.

## Clean alert headers and descriptions

We notice that the textual content of an alert is sometimes split between the header and the description. We will clean and combine these two fields for easier processing.

In [224]:
def clean_whitespace(text):
    ''' Clean newlines and extra whitespace in text '''
    if text is None:
        return None
    no_newlines = text.replace('\n', ' ')
    return re.sub(r'\s+', ' ', no_newlines).strip()

In [225]:
nyct["Header"] = nyct["Header"].apply(clean_whitespace)
nyct["Description"] = nyct["Description"].apply(clean_whitespace)

In [226]:
def combine_strings_or_nones(strings, delimiter=" "):
    ''' Join multiple strings, ignoring Nones '''
    return delimiter.join(list(filter(None, strings)))

In [227]:
nyct["Combined description"] = nyct.apply(
    lambda x: combine_strings_or_nones([x["Header"], x["Description"]]), 1)

In [228]:
nyct["Combined description"]

9         Northbound E F trains are holding in stations ...
10        Southbound 6 trains are delayed while we addre...
16        Northbound N trains are delayed while we remov...
24        Northbound 2 5 trains are delayed while we add...
29        Northbound F trains are delayed while we condu...
                                ...                        
343788    Southbound 4 trains have resumed making expres...
343792    Southbound 2 and 3 trains are proceeding at no...
343794    Southbound 6 trains are proceeding at normal s...
343800    3 trains are running with delays in both direc...
343801    3 trains are proceeding with delays in both di...
Name: Combined description, Length: 128197, dtype: object

## Identify delay causes from alert descriptions

In this section, we will use [regular expressions](https://en.wikipedia.org/wiki/Regular_expression) to parse the most likely issue for the delay, according to the alert description. This approach is not infallible, but as we will see, it can work remarkably well due to the fact that alert descriptions often use very reliable language. Using an LLM to determine the cause is also certainly an option, and I experimented with it, but ultimately decided against it. For more details check out my write-up!

Here we define the issue type and specific issues that we will use when categorizing alerts. How did I arrive at this list of issues? I landed on them using an iterative approach:

1. Define a few obvious issues that appear in the alerts, like "disruptive passenger" or "door problem".
2. Tag alerts with this keyword in the description
3. Filter by alerts that did not match any issue
4. Identify the most prevalent issues in those unmatched alerts and add them to the list
6. Repeat steps 2-4 until satisfied with the coverage
7. Come up with broader categories to group issues together (issue types)

In [229]:
class IssueType(StrEnum):
    MAINTENANCE = auto()
    BRAKE_ACTIVATED = auto()
    MECHANICAL_ISSUE = auto()
    HUMAN_DISRUPTION = auto()
    OBJECT_ON_TRACKS = auto()
    EMS_NYPD_FDNY_RESPONSE = auto()
    CLEANING = auto()
    MISC = auto()

In [230]:
class Issue(StrEnum):
    # Maintenance
    TRACK_MAINTENANCE = auto()
    SIGNAL_MAINTENANCE = auto()
    SWITCH_MAINTENANCE = auto()
    WORK_TRAIN = auto()
    MISC_MAINTENANCE = auto()
    # Brake activated
    BRAKES_ACTIVATED = auto()
    EMERGENCY_BRAKE_PULLED = auto()
    # Mechanical issue
    SIGNAL_PROBLEM = auto()
    SWITCH_PROBLEM = auto()
    COMMS_PROBLEM = auto()
    LOSS_OF_POWER = auto()
    RAIL_PROBLEM = auto()
    DOOR_PROBLEM = auto()
    TRAIN_PROBLEM = auto()
    MECHANICAL_PROBLEM = auto()
    # Human disruption
    DISRUPTIVE_PASSENGER = auto()
    PERSON_ON_TRACKS = auto()
    PERSON_STRUCK_BY_TRAIN = auto()
    MEDICAL_EMERGENCY = auto()
    VANDALISM = auto()
    # Object on tracks
    SOMETHING_ON_TRACKS = auto()
    FALLEN_TREE = auto()
    # EMS / NYPD / FDNY response
    EMS = auto()
    NYPD = auto()
    FDNY = auto()
    # Cleaning
    CLEANING = auto()
    # Misc
    SHORT_STAFFED = auto()
    FIRE = auto()
    FLOODING = auto()
    SOUTH_CHANNEL_BRIDGE = auto()
    TRACK_INSPECTIONS = auto()

In [231]:
# Match issues with issue type
ISSUE_TO_ISSUE_TYPE = {
    # Maintenance
    Issue.TRACK_MAINTENANCE: IssueType.MAINTENANCE,
    Issue.SIGNAL_MAINTENANCE: IssueType.MAINTENANCE,
    Issue.SWITCH_MAINTENANCE: IssueType.MAINTENANCE,
    Issue.WORK_TRAIN: IssueType.MAINTENANCE,
    Issue.MISC_MAINTENANCE: IssueType.MAINTENANCE,
    # Brake activated
    Issue.BRAKES_ACTIVATED: IssueType.BRAKE_ACTIVATED,
    Issue.EMERGENCY_BRAKE_PULLED: IssueType.BRAKE_ACTIVATED,
    # Mechanical issue
    Issue.SIGNAL_PROBLEM: IssueType.MECHANICAL_ISSUE,
    Issue.SWITCH_PROBLEM: IssueType.MECHANICAL_ISSUE,
    Issue.COMMS_PROBLEM: IssueType.MECHANICAL_ISSUE,
    Issue.LOSS_OF_POWER: IssueType.MECHANICAL_ISSUE,
    Issue.RAIL_PROBLEM: IssueType.MECHANICAL_ISSUE,
    Issue.DOOR_PROBLEM: IssueType.MECHANICAL_ISSUE,
    Issue.TRAIN_PROBLEM: IssueType.MECHANICAL_ISSUE,
    Issue.MECHANICAL_PROBLEM: IssueType.MECHANICAL_ISSUE,
    # Human disription
    Issue.DISRUPTIVE_PASSENGER: IssueType.HUMAN_DISRUPTION,
    Issue.PERSON_ON_TRACKS: IssueType.HUMAN_DISRUPTION,
    Issue.PERSON_STRUCK_BY_TRAIN: IssueType.HUMAN_DISRUPTION,
    Issue.MEDICAL_EMERGENCY: IssueType.HUMAN_DISRUPTION,
    Issue.VANDALISM: IssueType.HUMAN_DISRUPTION,
    # Object on tracks
    Issue.SOMETHING_ON_TRACKS: IssueType.OBJECT_ON_TRACKS,
    Issue.FALLEN_TREE: IssueType.OBJECT_ON_TRACKS,
    # EMS / NYPD / FDNY response
    Issue.EMS: IssueType.EMS_NYPD_FDNY_RESPONSE,
    Issue.NYPD: IssueType.EMS_NYPD_FDNY_RESPONSE,
    Issue.FDNY: IssueType.EMS_NYPD_FDNY_RESPONSE,
    # Cleaning
    Issue.CLEANING: IssueType.CLEANING,
    # Misc
    Issue.SHORT_STAFFED: IssueType.MISC,
    Issue.FIRE: IssueType.MISC,
    Issue.FLOODING: IssueType.MISC,
    Issue.SOUTH_CHANNEL_BRIDGE: IssueType.MISC,
    Issue.TRACK_INSPECTIONS: IssueType.MISC,
}

In [232]:
def get_issue_category(issue_str):
    ''' Given an issue in string format, return the issue type in string format '''
    if not issue_str:
        return None
    return ISSUE_TO_ISSUE_TYPE[Issue[issue_str]].name

In [233]:
# For convenience, we will define a named tuple to hold the issue and its associated regex pattern together.
IssueRegexPattern = namedtuple('IssueRegexPattern', ['issue', 'pattern'])

Here we define the regex pattern for each issue. Since our algorithm will choose the first pattern in the list that matches the description, **the order here matters!**. For that reason, we put issues that are more likely the root cause of the delay near the top (e.g. disruptive passenger), and issues that are more like fallbacks (e.g. NYPD) toward the end.

In [234]:
issue_regex_patterns = [
    IssueRegexPattern(Issue.SOUTH_CHANNEL_BRIDGE, r"(marine traffic|south channel bridge)"),
    IssueRegexPattern(Issue.FALLEN_TREE, r"tree"),
    IssueRegexPattern(Issue.FIRE, r"(fire|smoke)"),
    IssueRegexPattern(Issue.FLOODING, r"flooding|water"),
    IssueRegexPattern(Issue.DISRUPTIVE_PASSENGER, r"disruptive|assault|altercation"),
    IssueRegexPattern(Issue.DOOR_PROBLEM, r"door"),
    IssueRegexPattern(Issue.PERSON_STRUCK_BY_TRAIN, r"struck by[^\.]+train"),
    IssueRegexPattern(Issue.PERSON_ON_TRACKS, r"(unauthorized (person|individual)|fell onto the tracks)"),
    IssueRegexPattern(Issue.SOMETHING_ON_TRACKS, r"(debris|remov[^\.]+from the tracks)"),
    IssueRegexPattern(Issue.MEDICAL_EMERGENCY, r"(injured|medical|sick|((person|someone) in (need|crisis))|(needs help))"),
    IssueRegexPattern(Issue.EMERGENCY_BRAKE_PULLED, r"brake cord"),
    IssueRegexPattern(Issue.VANDALISM, r"vandaliz"),
    IssueRegexPattern(Issue.TRACK_MAINTENANCE, r"track maintenance"),
    IssueRegexPattern(Issue.SIGNAL_MAINTENANCE, r"signal maintenance"),
    IssueRegexPattern(Issue.SWITCH_MAINTENANCE, r"switch maintenance"),
    IssueRegexPattern(Issue.SIGNAL_PROBLEM, r"signal"),
    IssueRegexPattern(Issue.SWITCH_PROBLEM, r"switch"),
    IssueRegexPattern(Issue.COMMS_PROBLEM, r"communication[^\.]+(problem|malfunction|issue)"),
    IssueRegexPattern(Issue.RAIL_PROBLEM, r"(broken rail|replac[^\.]+rail|rail repair)"),
    IssueRegexPattern(Issue.LOSS_OF_POWER, r"power"),
    IssueRegexPattern(Issue.BRAKES_ACTIVATED, r"(train[^\.]+brake[^\.]+activat|activat[^\.]+train[^\.]+brake)"),
    IssueRegexPattern(Issue.CLEANING, r"cleaning"),
    IssueRegexPattern(Issue.WORK_TRAIN, r"work train"),
    IssueRegexPattern(Issue.MISC_MAINTENANCE, r"(maintenance|planned (track )?work|work equipment)"),
    IssueRegexPattern(Issue.TRAIN_PROBLEM, r"(removed? a train.*from service|remov.*train.*in need of repair)"),
    IssueRegexPattern(Issue.TRACK_INSPECTIONS, r"track inspection"),
    IssueRegexPattern(Issue.MECHANICAL_PROBLEM, r"mechanical problems?"),
    IssueRegexPattern(Issue.SHORT_STAFFED, r"with the crews we have available"),
    IssueRegexPattern(Issue.EMS, r"\bems\b"),
    IssueRegexPattern(Issue.NYPD, r"nypd"),
    IssueRegexPattern(Issue.FDNY, r"fdny"),
]

In [235]:
def find_first_issue(text, issue_patterns):
    ''' Given text and a list of IssueRegexPattern, return the first issue with a match '''
    for issue_pattern in issue_patterns:
        if re.search(issue_pattern.pattern, text.lower()) is not None:
            return issue_pattern.issue.name
    return None

In [236]:
nyct["Issue"] = nyct["Combined description"].apply(lambda x: find_first_issue(x, issue_regex_patterns))

In [237]:
nyct["Issue type"] = nyct["Issue"].apply(get_issue_category)

In [238]:
nyct

,Alert ID,Event ID,Update Number,Date,Agency,Status Label,Affected,Header,Description,Combined description,Issue,Issue type
9,333837,161166,0,07/18/2024 06:17:00 AM,NYCT Subway,delays,E | F,Northbound E F trains are holding in stations ...,None,Northbound E F trains are holding in stations ...,RAIL_PROBLEM,MECHANICAL_ISSUE
10,332834,160668,0,07/15/2024 10:53:00 AM,NYCT Subway,delays,6,Southbound 6 trains are delayed while we addre...,None,Southbound 6 trains are delayed while we addre...,SIGNAL_PROBLEM,MECHANICAL_ISSUE
16,330046,159187,0,07/04/2024 09:29:00 PM,NYCT Subway,delays,N,Northbound N trains are delayed while we remov...,None,Northbound N trains are delayed while we remov...,SOMETHING_ON_TRACKS,OBJECT_ON_TRACKS
24,326100,157189,0,06/21/2024 05:59:00 PM,NYCT Subway,delays,2 | 5,Northbound 2 5 trains are delayed while we add...,None,Northbound 2 5 trains are delayed while we add...,DOOR_PROBLEM,MECHANICAL_ISSUE
29,322274,155232,0,06/07/2024 07:37:00 PM,NYCT Subway,delays,F,Northbound F trains are delayed while we condu...,None,Northbound F trains are delayed while we condu...,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...
343788,118,87,4,04/28/2020 08:51:00 PM,NYCT Subway,delays,4,Southbound 4 trains have resumed making expres...,None,Southbound 4 trains have resumed making expres...,SIGNAL_PROBLEM,MECHANICAL_ISSUE
343792,117,86,3,04/28/2020 08:45:00 PM,NYCT Subway,delays,2 | 3,Southbound 2 and 3 trains are proceeding at no...,None,Southbound 2 and 3 trains are proceeding at no...,SIGNAL_PROBLEM,MECHANICAL_ISSUE
343794,115,85,1,04/28/2020 08:09:00 PM,NYCT Subway,delays,6,Southbound 6 trains are proceeding at normal s...,None,Southbound 6 trains are proceeding at normal s...,SIGNAL_PROBLEM,MECHANICAL_ISSUE
343800,86,82,0,04/28/2020 02:38:00 PM,NYCT Subway,delays,3,3 trains are running with delays in both direc...,None,3 trains are running with delays in both direc...,BRAKES_ACTIVATED,BRAKE_ACTIVATED


Let's manually inspect some of these classifications to make sure they seem reasonable.

In [239]:
def filter_by_issue_or_type(df, enum_value, column):
    ''' Return the rows in df where the column matches the enum value; enum_value can be None '''
    if (enum_value is None):
        return df[df[column].isna()]
    else:
        return df[df[column] == enum_value]

In [240]:
def filter_by_issue(df, issue):
    return filter_by_issue_or_type(df, issue, "Issue")

In [241]:
def print_as_list(df):
    for row in list(df):
        print(row, end="\n\n")

Let's print the descriptions of some delays tagged as switch problems:

In [242]:
print_as_list(filter_by_issue(nyct, Issue.SWITCH_PROBLEM.name).sample(10)["Combined description"])

Southbound E trains have resumed making express stops from Jackson Hts-Roosevelt Av to Queens Plaza after our crews corrected a switch problem at 36 St. Continue to expect delays in E F M R service as we get trains back on schedule.

2 trains are delayed in both directions because of a switch problem at Chambers St.

Southbound A trains are running on the local track from 168 St to 145 St while we address a switch problem at 168 St. You may wait longer for a C train while trains share the local tracks.

Southbound 4 5 trains are running with delays after we addressed a switch malfunction near 149 St-Grand Concourse.

Southbound A B C D trains are running with extensive delays in Manhattan. Avoid these lines and take 1 2 3 4 trains or M7/M10 buses. Southbound D trains are running local from 125 St to 59 St-Columbus Circle. You may wait longer for northbound A B C D service. What's Happening? Our crews are on the scene at 59 St-Columbus Circle addressing a switch problem. Alternatives: F

And some delays where the issue is an object on the tracks:

In [243]:
print_as_list(filter_by_issue(nyct, Issue.SOMETHING_ON_TRACKS.name).sample(10)["Combined description"])

Southbound 5 trains are delayed because of debris on the track that caused a train's brakes to activate at Morris Park.

A Rockaway Park Shuttle trains are running with delays in both directions after we removed debris from the tracks which caused a train's brakes to activate near Broad Channel.

Southbound E trains are delayed while our crews work to remove debris from the tracks at 65 St.

Expect long waits for northbound F trains in Queens after we removed a train that had its brakes activated by debris on the tracks from service at Jay St-MetroTech.

Southbound N Q R W trains are disrupted while we remove debris from the track area at 57 St-7 Av. Some southbound Q trains are running on the F line from Lexington Av/63 St to 34 St-Herald Sq, then along the B line to DeKalb Av. Some southbound R trains are running on the F line from 36 St to 34 St-Herald Sq, then along the B line to DeKalb Av. Some southbound F trains are running on the E line from Jackson Hts-Roosevelt Av to Queens P

And finally, let's sample a few alerts with no issue identified.

In [244]:
print_as_list(filter_by_issue(nyct, None).sample(10)["Combined description"])

The 7:03 AM SIR from St George Terminal to Great Kills will not run. The 7:01 AM SIR from St George Terminal to Tottenville will make all local stops. We're running as much service as we can with the equipment we have available.

Southbound B D trains are running with delays after our crews safely moved equipment on the Manhattan Bridge.

Expect a longer wait for Brighton Beach-bound B and Coney Island-bound Q trains after we removed a Q train from service at DeKalb Av.

Northbound =4 train service has resumed regular service on the local track from Franklin Av-Medgar Evers College to Atlantic Av-Barclays Ctr after our crews safely moved equipment between Atlantic Av-Barclays Ctr and Borough Hall.

Expect longer waits for 6 trains. We're running as much service as we can with the train crews we have available.

The 5:26 AM SIR trip from Tottenville to St George will not run. We are running as much service as we can with the train crews we have available.

Expect longer waits for B trai

You may notice that some alerts *could* be classified as one of our identified issues, but weren't correctly picked up by our regex patterns. This is okay; as we'll soon see that our coverage is still high enough to work with. More advanced NLP techniques could help, but we've reached the [80/20 point](https://en.wikipedia.org/wiki/Pareto_principle).

## Identify the main station associated with the alert

Next, we'll want to identify which station the issue occurred near or at. We'll start by creating a regex pattern that matches *all* station names.

In [245]:
STATION_NAMES_DEDUPED = set(list(stations_df["Stop Name"]))
# Escape parentheses so they aren't interpreted as regex groups
ESCAPED_STATION_NAMES = [station_name.replace('(',r'\(').replace(')',r'\)') for station_name in STATION_NAMES_DEDUPED]
STATION_NAMES_REGEX = fr'({"|".join(ESCAPED_STATION_NAMES)})'

In [246]:
def clean_text_for_station_identification(text):
    ''' Perform some simple cleaning to help identify station names '''
    return text \
        .replace(' - ', '-') \
        .replace('Jct', 'Junction') \
        .replace('Delancy', 'Delancey') \
        .replace("Christopher St-Sheridan Sq", "Christopher St-Stonewall")

In [247]:
def get_station(description):
    ''' Given some text, find a station name that follows "at", "near", or "for" '''
    if not description:
        return None
    description_cleaned = clean_text_for_station_identification(description)
    if "at that station" in description:
        station_name_regex = STATION_NAMES_REGEX
        regex_group = 1
    else:
        station_name_regex = fr'(at|near|for) {STATION_NAMES_REGEX}'
        regex_group = 2
    station_name_match = re.search(station_name_regex, description_cleaned)
    if station_name_match:
        return station_name_match.group(regex_group)
    else:
        return None

In [248]:
nyct["Station"] = nyct["Combined description"].apply(get_station)

In [249]:
nyct["Station"].value_counts()

Station
59 St                       3011
125 St                      2826
14 St-Union Sq              2244
Times Sq-42 St              2103
Atlantic Av-Barclays Ctr    2033
                            ... 
Pleasant Plains                4
New Dorp                       4
Bay Terrace                    3
Tompkinsville                  2
Arthur Kill                    2
Name: count, Length: 369, dtype: int64

Let's inspect a few descriptions where no station was identified.

In [250]:
no_station_identified = nyct[nyct["Station"].isna()]
print_as_list(no_station_identified.sample(10)["Combined description"])

You may experience a longer wait for northbound 4 and 5 trains. We're running as many trains as we can with the train crews we have available.

Expect longer wait times for A D N Q trains in both directions. We're running as much service as we can with the train crews we have available.

A Rockaway Park Shuttle trains are running with delays in both directions after the South Channel Bridge opened to allow marine traffic to pass.

You may experience a longer wait for a northbound F train. We're running as much service as we can with the train crews we have available.

SIR are running with delays due to flooding conditions between Pleasant Plains and Tottenville in both directions.

You may wait longer for a C train in both directions after we removed a train with a door problem from service earlier in Manhattan and requested medical help for someone earlier in Brooklyn.

The 5:11 PM St George to Tottenville SIR trip did not run. The 6:09 PM Tottenville to St George SIR trip will not ru

One failure mode of our algorithm is alerts which use the phrasing `"... <station name> ... at that station."`. We will also leave this as a potential 80/20 improvement.

### Disambiguate station names by services

We have station names, but *which* station? I mean, how many 23 Sts are there?

In [251]:
len([station for station in stations_df["Stop Name"] if station == "23 St"])

5

I rest my case. In order to disambiguate which station each alert refers to, we can use the `"Affected"` column in the alerts dataset to determine which line the station is on, and then find the station which matches the name and line.

Why not just compare the `"Affected"` services in the alert to the `"Daytime Routes"` listed for each station? Take this alert for example:

In [252]:
nyct[(nyct["Affected"] == "A") & (nyct["Station"] == "23 St")].sample(1)

,Alert ID,Event ID,Update Number,Date,Agency,Status Label,Affected,Header,Description,Combined description,Issue,Issue type,Station
306818,36366,8641,1,10/20/2020 02:14:00 AM,NYCT Subway,delays,A,Southbound A trains have resumed regular servi...,None,Southbound A trains have resumed regular servi...,BRAKES_ACTIVATED,BRAKE_ACTIVATED,23 St


In this case, the issue at 23 St affects service on the `A`, but 23 St does not serve the `A` train! We can remedy this by looking at *lines* instead of individual stations. Our approach will be to group stations by line, and determine which services run on that line, as well as all the station names. Then, for a given station name and affected services, we match it to a line which includes that station name and contains that service. If this approach fails for whatever reason, we will pick a station at random whose name matches.

One edge case arises in which a line contains a station which serves a route that does not run along the rest of the line. One example is BMT Broadway - Brighton, which includes West 8 St-NY Aquarium, which serves the `F`, despite the `F` not running along that line. This could cause issues because when searching for a train station called `"23 St"` that serves the `F`, the obvious choice would be the 23 St on the IND 6th Av - Culver, but the 23 St on BMT Broadway - Brighton would technically also be a candidate by our previous logic. We mitigate this issue by requiring that a service must be served by at least 3 stations on the line for it to "count".

One note is that we only have the *daytime* routes of each station, which excludes late-night service changes. However, this is a fine approximation with the data available.

In [253]:
# The "Affected" column in the alerts dataset is pipe-delimited
def split_affected_services(services_string):
    return services_string.split(' | ')

# The "Daytime Routes" column in the station dataset is space-delimited
def split_served_services(services_string):
    return services_string.split(' ')

In [254]:
def flatten_daytime_routes(daytime_routes_list, min_appearances=3):
    ''' Given a list of pipe-delimited services, flatten them into a set
        Also filters out services that appear fewer than `min_appearances` times '''
    # If the line has < min_appearances stations, relax the restriction
    min_appearances = min(len(daytime_routes_list), min_appearances)
    routes_list = []
    for daytime_routes in daytime_routes_list:
        routes_list += split_served_services(daytime_routes)
    counter = Counter(routes_list)
    return set([key for key in counter if counter[key] >= min_appearances])

In [255]:
services_per_line = stations_df.groupby(["Line", "Division"]).aggregate(**{
    "Stop Name": ("Stop Name", list), 
    "GTFS Stop ID": ("GTFS Stop ID", list),
    "Daytime Routes": ("Daytime Routes", flatten_daytime_routes)
}).reset_index()
services_per_line.head(5)

,Line,Division,Stop Name,GTFS Stop ID,Daytime Routes
0,4th Av,BMT,"[Atlantic Av-Barclays Ctr, Union St, 4 Av-9 St...","[R31, R32, R33, R34, R35, R36, R39, R40, R41, ...","{R, N}"
1,63rd St,IND,"[21 St-Queensbridge, Roosevelt Island, Lexingt...","[B04, B06, B08]",{F}
2,6th Av - Culver,IND,"[W 4 St-Wash Sq, 57 St, 47-50 Sts-Rockefeller ...","[D20, B10, D15, D16, D17, D18, D19, D21, D22, ...","{F, G, B, D, M}"
3,8th Av - Fulton St,IND,"[Inwood-207 St, Dyckman St, 190 St, 181 St, 17...","[A02, A03, A05, A06, A07, A09, A10, A11, A12, ...","{B, E, C, A}"
4,Astoria,BMT,"[Astoria-Ditmars Blvd, Astoria Blvd, 30 Av, Br...","[R01, R03, R04, R05, R06, R08, R11, R13, R09]","{N, W}"


Let's verify that a line with fewer than 3 stops still contains routes:

In [256]:
services_per_line[services_per_line["Line"] == "Lexington - Shuttle"]

,Line,Division,Stop Name,GTFS Stop ID,Daytime Routes
19,Lexington - Shuttle,IRT,"[Times Sq-42 St, Grand Central-42 St]","[902, 901]",{S}


In [257]:
def does_line_match_name_and_services(line, station_name, services):
    ''' Ensure the station name matches a station in the line and ensure at least one station in the affected routes runs on that line '''
    return len(set(services).intersection(line["Daytime Routes"])) > 0 and station_name in line["Stop Name"]

In [258]:
def fallback_disambiguate_station_name(station_name):
    ''' Pick a random station that matches the station name '''
    return stations_df[stations_df["Stop Name"] == station_name].sample(1)["GTFS Stop ID"].iloc[0]

In [259]:
def disambiguate_station_name_by_line(station_name, affected_services):
    ''' Returns the GTFS Stop ID for the station that matches the affected services
        Resorts to randomly picking a station with a matching name as a fallback '''
    if not station_name or not affected_services:
        return None
    affected_services = split_affected_services(affected_services)
    matching_lines = services_per_line[services_per_line.apply(
        lambda line: does_line_match_name_and_services(line, station_name, affected_services), axis=1)]
    if len(matching_lines) == 0:
        return fallback_disambiguate_station_name(station_name)
    matching_line = matching_lines.sample(1).iloc[0]
    # Guaranteed to exist because of the filter
    matching_stop_name_index = list(matching_line["Stop Name"]).index(station_name)
    return matching_line["GTFS Stop ID"][matching_stop_name_index]

This line can take a while to run.

In [260]:
nyct["GTFS Stop ID"] = nyct.apply(lambda x: disambiguate_station_name_by_line(x["Station"], x["Affected"]), axis=1)

Let's inspect some results by looking at alerts for `"23 St"` and ensure there are different Station IDs present.

In [261]:
nyct[nyct["Station"] == "23 St"][["Affected", "Station", "GTFS Stop ID"]].sample(10)

,Affected,Station,GTFS Stop ID
56159,6,23 St,634
95487,F,23 St,D18
124736,N | Q,23 St,R19
79926,A | C | E,23 St,A30
193650,F | M,23 St,D18
287678,R | W,23 St,R19
49671,R | W,23 St,R19
162203,C | E,23 St,A30
305844,1,23 St,130
69335,N | Q | R | W,23 St,R19


And to verify one of those GTFS Stop IDs:

In [262]:
stations_df[stations_df["GTFS Stop ID"] == "A30"]

,GTFS Stop ID,Station ID,Complex ID,Division,Line,Stop Name,Borough,CBD,Daytime Routes,Structure,GTFS Latitude,GTFS Longitude,North Direction Label,South Direction Label,ADA,ADA Northbound,ADA Southbound,ADA Notes,Georeference
164,A30,165,165,IND,8th Av - Fulton St,23 St,M,True,C E,Subway,40.745906,-73.998041,Uptown,Downtown,0,0,0,NaN,POINT (-73.998041 40.745906)


## Group multiple alerts for the same event together

Now, we can take the penultimate step; grouping events together by the `Event ID`. Each alert is tied to an `Event ID` along with an `Update Number` that starts at `0` and increases with each update. Because we would like to analyze *events*, we will group by the `Event ID` and aggregate the remaining relevant data.

In [263]:
def parse_alert_datetime(alert_datetime):
    ''' Converts a datetime string to a Python datetime object '''
    return datetime.strptime(alert_datetime, "%m/%d/%Y %I:%M:%S %p")

def parse_alert_datetimes(alert_datetimes):
    ''' Converts a list of datetime strings to Python datetime objects '''
    return [parse_alert_datetime(dt) for dt in alert_datetimes]

def aggregate_items_dedup(lists_of_items):
    ''' Flatten a list of lists to a deduplicated set '''
    items_set = set()
    for list_of_items in lists_of_items:
        items_set = items_set.union(set(list_of_items))
    return list(items_set)

def group_affected_services(list_of_affected_services):
    ''' Aggregate affected services by deduplicating '''
    return aggregate_items_dedup([split_affected_services(affected) for affected in list_of_affected_services])

In [264]:
# First we sort by update number, so that when we choose the first issue / stop ID, it is always from the earliest alert
nyct_events_groupby = nyct.sort_values("Update Number").groupby("Event ID")
nyct_events = nyct_events_groupby.agg(**{
    'First alert datetime': ('Date', lambda x: min(parse_alert_datetimes(x))),
    'Last alert datetime': ('Date', lambda x: max(parse_alert_datetimes(x))),
    'Number of updates': ('Update Number', 'count'),
    'Affected services': ('Affected', group_affected_services),
    'Combined description': ('Combined description', list),
    'Issue': ('Issue', 'first'),
    'Issue type': ('Issue type', 'first'),
    'GTFS Stop ID': ('GTFS Stop ID', 'first'),
}).reset_index()

In [265]:
nyct_events.sample(10)

,Event ID,First alert datetime,Last alert datetime,Number of updates,Affected services,Combined description,Issue,Issue type,GTFS Stop ID
32936,87040,2022-12-13 21:10:00,2022-12-13 21:10:00,1,"[3, 2]",[Southbound 2 3 trains are running with delays...,DOOR_PROBLEM,MECHANICAL_ISSUE,127
7405,16171,2021-02-18 22:51:00,2021-02-18 23:11:00,4,"[D, A]",[Southbound A and D trains are delayed while o...,BRAKES_ACTIVATED,BRAKE_ACTIVATED,A16
37902,97095,2023-03-18 00:08:00,2023-03-18 00:13:00,2,[E],[E trains are delayed entering/leaving World T...,MEDICAL_EMERGENCY,HUMAN_DISRUPTION,E01
30359,80830,2022-10-20 20:54:00,2022-10-20 21:06:00,3,"[E, C]",[Southbound C E trains are delayed while we re...,TRAIN_PROBLEM,MECHANICAL_ISSUE,A34
50127,126789,2023-11-06 10:17:00,2023-11-06 10:17:00,1,"[E, F]",[Northbound E F trains are running with delays...,PERSON_ON_TRACKS,HUMAN_DISRUPTION,G19
1316,2832,2020-07-03 19:30:00,2020-07-03 23:16:00,3,"[3, 2]",[Downtown 2 3 trains are delayed because someo...,PERSON_STRUCK_BY_TRAIN,HUMAN_DISRUPTION,225
27495,72896,2022-08-16 09:54:00,2022-08-16 09:54:00,1,[1],[1 trains are running with delays in both dire...,PERSON_ON_TRACKS,HUMAN_DISRUPTION,118
38178,97636,2023-03-22 18:02:00,2023-03-22 18:18:00,2,[6],[Southbound 6 trains are delayed while we requ...,MEDICAL_EMERGENCY,HUMAN_DISRUPTION,631
46214,116710,2023-08-22 07:05:00,2023-08-22 07:08:00,2,"[N, D]",[Southbound N trains are delayed. Southbound D...,TRACK_MAINTENANCE,MAINTENANCE,B12
4207,9358,2020-11-01 02:34:00,2020-11-01 02:44:00,2,"[6, 4]",[Northbound 4 and 6 trains are delayed while w...,DISRUPTIVE_PASSENGER,HUMAN_DISRUPTION,635


In [266]:
def format_percent(ratio):
    return "{:.2%}".format(ratio)

In [267]:
no_issue_identified = filter_by_issue_or_type(nyct_events, None, "Issue")
print(f'Percent of events with an identified issue: {format_percent(1 - len(no_issue_identified) / len(nyct_events))}')

Percent of events with an identified issue: 96.36%


In [268]:
no_station_identified = nyct_events[nyct_events["GTFS Stop ID"].isna()]
print(f'Percent of events with an identified station: {format_percent(1 - len(no_station_identified) / len(nyct_events))}')

Percent of events with an identified station: 91.27%


In [269]:
percent_of_stations_represented = len(nyct_events["GTFS Stop ID"].value_counts()) / len(stations_df)
print(f'Percent of stations represented: {format_percent(percent_of_stations_represented)}')

Percent of stations represented: 97.58%


## Join with station data

Now, we can complete our data preparation by joining each event with its associated station, if it was identified with one.

In [270]:
# Perform a left join so that we don't drop any alerts.
nyct_events_station = nyct_events.join(stations_df.set_index("GTFS Stop ID"), on="GTFS Stop ID", how="left")
# Drop columns that seem less relevant.
nyct_events_station = nyct_events_station.drop(
    columns=["Station ID", "Complex ID", "CBD", "North Direction Label", "South Direction Label", "ADA Northbound", "ADA Southbound", 
             "ADA Notes", "Georeference"])
nyct_events_station["ADA"] = nyct_events_station["ADA"].apply(lambda x: np.nan if pd.isna(x) else bool(x))

In [271]:
nyct_events_station.sample(10)

,Event ID,First alert datetime,Last alert datetime,Number of updates,Affected services,Combined description,Issue,Issue type,GTFS Stop ID,Division,Line,Stop Name,Borough,Daytime Routes,Structure,GTFS Latitude,GTFS Longitude,ADA
45272,114279,2023-08-02 23:25:00,2023-08-02 23:33:00,2,"[B, Q]",[Southbound B trains are running local from Pr...,NYPD,EMS_NYPD_FDNY_RESPONSE,F27,IND,6th Av - Culver,Church Av,Bk,F G,Subway,40.644041,-73.979678,True
467,1083,2020-05-19 02:02:00,2020-05-19 02:12:00,2,[A],[Southbound A trains are delayed while we addr...,SIGNAL_PROBLEM,MECHANICAL_ISSUE,A15,IND,8th Av - Fulton St,125 St,M,A C B D,Subway,40.811109,-73.952343,True
31960,84890,2022-11-25 12:27:00,2022-11-25 12:30:00,2,[7],[Flushing-bound 7 trains are delayed while we ...,MECHANICAL_PROBLEM,MECHANICAL_ISSUE,718,IRT,Flushing,Queensboro Plaza,Q,7,Elevated,40.750582,-73.940202,False
31692,84339,2022-11-19 06:42:00,2022-11-19 06:42:00,1,[4],[Southbound 4 trains are running with delays a...,CLEANING,CLEANING,418,IRT,Lexington Av,Fulton St,M,4 5,Subway,40.710368,-74.009509,True
14161,31642,2021-09-17 22:55:00,2021-09-17 22:55:00,1,[L],[L trains are delayed in both directions in Ma...,NYPD,EMS_NYPD_FDNY_RESPONSE,L06,BMT,Canarsie,1 Av,M,L,Subway,40.730953,-73.981628,True
54524,136912,2024-01-20 02:37:00,2024-01-20 02:40:00,2,"[R, E, F]",[Southbound E F R trains are delayed while we ...,MECHANICAL_PROBLEM,MECHANICAL_ISSUE,G10,IND,Queens Blvd,63 Dr-Rego Park,Q,M R,Subway,40.729846,-73.861604,False
62463,154536,2024-06-04 03:04:00,2024-06-04 03:07:00,2,[6],[Southbound 6 trains are delayed while NYPD re...,DISRUPTIVE_PASSENGER,HUMAN_DISRUPTION,621,IRT,Lexington Av,125 St,M,4 5 6,Subway,40.804138,-73.937594,True
16961,41298,2021-12-07 03:32:00,2021-12-07 03:56:00,2,"[6, 5, 4]",[Downtown 6 trains are rerouted on the express...,SOMETHING_ON_TRACKS,OBJECT_ON_TRACKS,629,IRT,Lexington Av,59 St,M,4 5 6,Subway,40.762526,-73.967967,False
12141,26205,2021-07-17 10:39:00,2021-07-17 11:57:00,2,[Q],[Northbound Q trains are delayed due to a swit...,SWITCH_PROBLEM,MECHANICAL_ISSUE,D26,BMT,Broadway - Brighton,Prospect Park,Bk,B Q S,Open Cut,40.661614,-73.962246,True
43331,109591,2023-06-27 20:26:00,2023-06-27 23:05:00,5,[3],[3 trains are delayed in both directions while...,SIGNAL_PROBLEM,MECHANICAL_ISSUE,257,IRT,Eastern Pky,New Lots Av,Bk,3,Elevated,40.666235,-73.884079,False


# Finishing touches
There's just a few more things we can add to make analysis a little more convenient.

In [272]:
def simplify_service_name(service_name):
    # Express variants
    if service_name in ["FX", "6X", "7X"]:
        return service_name[0]
    # J and Z are often taken together
    elif service_name in ["J", "Z"]:
        return "JZ"
    # Shuttles; H = Rockaway Park, FS = Franklin, GS = 42nd St
    elif service_name in ("H", "FS", "GS"):
        return "S"
    return service_name

In [273]:
nyct_events_station["Affected services (basic)"] = nyct_events_station["Affected services"].apply(
    lambda services: [simplify_service_name(service) for service in services])

In [274]:
nyct_events_station["Month"] = nyct_events_station["First alert datetime"].apply(lambda x: x.month)
nyct_events_station["Year month"] = nyct_events_station["First alert datetime"].apply(lambda x: x.strftime("%Y-%m"))
nyct_events_station["Day of week"] = nyct_events_station["First alert datetime"].apply(lambda x: x.weekday())
nyct_events_station["Daytime Routes"] = nyct_events_station["Daytime Routes"].apply(
    lambda x: [] if pd.isna(x) else split_served_services(x))

In [275]:
nyct_events_station

,Event ID,First alert datetime,Last alert datetime,Number of updates,Affected services,Combined description,Issue,Issue type,GTFS Stop ID,Division,...,Borough,Daytime Routes,Structure,GTFS Latitude,GTFS Longitude,ADA,Affected services (basic),Month,Year month,Day of week
0,82,2020-04-28 14:38:00,2020-04-28 14:45:00,2,[3],[3 trains are running with delays in both dire...,BRAKES_ACTIVATED,BRAKE_ACTIVATED,301,IRT,...,M,[3],Subway,40.823880,-73.936470,False,[3],4,2020-04,1
1,85,2020-04-28 20:09:00,2020-04-28 20:09:00,1,[6],[Southbound 6 trains are proceeding at normal ...,SIGNAL_PROBLEM,MECHANICAL_ISSUE,612,IRT,...,Bx,[6],Elevated,40.826525,-73.886283,False,[6],4,2020-04,1
2,86,2020-04-28 20:45:00,2020-04-28 20:45:00,1,"[3, 2]",[Southbound 2 and 3 trains are proceeding at n...,SIGNAL_PROBLEM,MECHANICAL_ISSUE,132,IRT,...,M,"[1, 2, 3]",Subway,40.737826,-74.000201,False,"[3, 2]",4,2020-04,1
3,87,2020-04-28 20:51:00,2020-04-28 20:51:00,1,[4],[Southbound 4 trains have resumed making expre...,SIGNAL_PROBLEM,MECHANICAL_ISSUE,132,IRT,...,M,"[1, 2, 3]",Subway,40.737826,-74.000201,False,[4],4,2020-04,1
4,88,2020-04-28 20:40:00,2020-04-28 20:56:00,2,[A],[Northbound A trains are delayed while our cre...,DOOR_PROBLEM,MECHANICAL_ISSUE,A40,IND,...,Bk,"[A, C]",Subway,40.699337,-73.990531,False,[A],4,2020-04,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
67016,167851,2024-08-30 21:13:00,2024-08-30 21:27:00,2,[1],[Southbound 1 trains are delayed while we addr...,SIGNAL_PROBLEM,MECHANICAL_ISSUE,137,IRT,...,M,"[1, 2, 3]",Subway,40.715478,-74.009266,True,[1],8,2024-08,4
67017,167852,2024-08-30 21:20:00,2024-08-30 21:20:00,1,"[3, 2]",[Northbound 2 3 trains are running with delays...,DISRUPTIVE_PASSENGER,HUMAN_DISRUPTION,123,IRT,...,M,"[1, 2, 3]",Subway,40.778453,-73.981970,True,"[3, 2]",8,2024-08,4
67018,167853,2024-08-30 21:43:00,2024-08-30 21:48:00,2,"[C, A]",[Southbound A C trains are delayed while we re...,TRAIN_PROBLEM,MECHANICAL_ISSUE,A38,IND,...,M,"[A, C]",Subway,40.710197,-74.007691,True,"[C, A]",8,2024-08,4
67019,167855,2024-08-30 22:08:00,2024-08-30 22:08:00,1,[L],[8 Av-bound L trains are running with delays a...,BRAKES_ACTIVATED,BRAKE_ACTIVATED,L06,BMT,...,M,[L],Subway,40.730953,-73.981628,True,[L],8,2024-08,4


# Export

Now we'll prepare and export our hard work to analyze further in the next notebook. We'll first define human-friendly descriptions for each issue and issue type.

In [276]:
ISSUE_TO_TEXT = {
    # Maintenance
    Issue.TRACK_MAINTENANCE: "Track maintenance",
    Issue.SIGNAL_MAINTENANCE: "Signal maintenance",
    Issue.SWITCH_MAINTENANCE: "Switch maintenance",
    Issue.WORK_TRAIN: "Work train-related",
    Issue.MISC_MAINTENANCE: "Unspecified maintenance",
    # Brake activated
    Issue.BRAKES_ACTIVATED: "Brakes activated",
    Issue.EMERGENCY_BRAKE_PULLED: "Emergency brake cord pulled",
    # Mechanical issue
    Issue.SIGNAL_PROBLEM: "Signal problem",
    Issue.SWITCH_PROBLEM: "Switch problem",
    Issue.COMMS_PROBLEM: "Communication problem",
    Issue.LOSS_OF_POWER: "Power issue",
    Issue.RAIL_PROBLEM: "Rail problem",
    Issue.DOOR_PROBLEM: "Door problem",
    Issue.TRAIN_PROBLEM: "Unspecified train problem",
    Issue.MECHANICAL_PROBLEM: "Unspecified mechanical problem",
    # Human disription
    Issue.DISRUPTIVE_PASSENGER: "Disruptive passenger",
    Issue.PERSON_ON_TRACKS: "Person on tracks",
    Issue.PERSON_STRUCK_BY_TRAIN: "Person struck by train",
    Issue.MEDICAL_EMERGENCY: "Medical emergency",
    Issue.VANDALISM: "Vandalism",
    # Object on tracks
    Issue.SOMETHING_ON_TRACKS: "Something on tracks",
    Issue.FALLEN_TREE: "Fallen tree",
    # EMS / NYPD / FDNY response
    Issue.EMS: "Unspecified EMS response",
    Issue.NYPD: "Unspecified NYPD response",
    Issue.FDNY: "Unspecified FDNY response",
    # Cleaning
    Issue.CLEANING: "Cleaning",
    # Short staffed
    Issue.SHORT_STAFFED: "Short-staffed",
    # Misc
    Issue.FIRE: "Fire / smoke",
    Issue.FLOODING: "Flooding",
    Issue.SOUTH_CHANNEL_BRIDGE: "South Channel Bridge open",
    Issue.TRACK_INSPECTIONS: "Track inspection",
}

ISSUE_TYPE_TO_TEXT = {
    IssueType.MAINTENANCE: "Maintenance",
    IssueType.BRAKE_ACTIVATED: "Brake activated",
    IssueType.MECHANICAL_ISSUE: "Mechanical issue",
    IssueType.HUMAN_DISRUPTION: "Passenger issue",
    IssueType.OBJECT_ON_TRACKS: "Object on tracks",
    IssueType.EMS_NYPD_FDNY_RESPONSE: "EMS/NYPD/FDNY response",
    IssueType.CLEANING: "Cleaning",
    IssueType.MISC: "Miscellaneous"
}

In [277]:
def get_issue_text(issue_enum_str):
    if not issue_enum_str:
        return None
    return ISSUE_TO_TEXT[Issue[issue_enum_str]]

def get_issue_type_text(issue_type_enum_str):
    if not issue_type_enum_str:
        return None
    return ISSUE_TYPE_TO_TEXT[IssueType[issue_type_enum_str]]

# Export data

## Export in JSON format

Since JSON can handle arrays, we will export to that data format first. We will also rename the columns to remove the spaces, as this is more convenient for consumption in Javascript / Typescript.

In [278]:
def to_snake_case(text):
    return text.lower().replace(" ", "_").replace('(', '').replace(')', '')

In [279]:
rename_map = dict([[name, to_snake_case(name)] for name in list(nyct_events_station.columns)])

In [280]:
rename_map['Number of updates'] = 'num_updates'
rename_map['Combined description'] = 'alert_descriptions'
rename_map['Issue'] = 'delay_issue'
rename_map['Issue type'] = 'delay_issue_category'
rename_map['GTFS Stop ID'] = 'station_gtfs_stop_id'
rename_map['Division'] = 'station_division'
rename_map['Line'] = 'station_line'
rename_map['Stop Name'] = 'station_name'
rename_map['Borough'] = 'station_borough'
rename_map['Daytime Routes'] = 'station_daytime_routes'
rename_map['Structure'] = 'station_structure_type'
rename_map['GTFS Latitude'] = 'station_latitude'
rename_map['GTFS Longitude'] = 'station_longitude'
rename_map['ADA'] = 'is_ada_friendly'

In [281]:
def convert_to_json(df):
    return json.loads(df.to_json(orient='records'))

In [282]:
nyct_events_renamed = nyct_events_station.rename(rename_map, axis=1)
nyct_events_json = convert_to_json(nyct_events_renamed)

In [283]:
print(json.dumps(nyct_events_json[0], indent=2))

{
  "event_id": 82,
  "first_alert_datetime": 1588084680000,
  "last_alert_datetime": 1588085100000,
  "num_updates": 2,
  "affected_services": [
    "3"
  ],
  "alert_descriptions": [
    "3 trains are running with delays in both directions while we investigate why a train's brakes were activated at Harlem - 148 St.",
    "3 trains are proceeding with delays in both directions after we moved a train that had its brakes activated at Harlem - 148 St."
  ],
  "delay_issue": "BRAKES_ACTIVATED",
  "delay_issue_category": "BRAKE_ACTIVATED",
  "station_gtfs_stop_id": "301",
  "station_division": "IRT",
  "station_line": "Lenox - White Plains Rd",
  "station_name": "Harlem-148 St",
  "station_borough": "M",
  "station_daytime_routes": [
    "3"
  ],
  "station_structure_type": "Subway",
  "station_latitude": 40.82388,
  "station_longitude": -73.93647,
  "is_ada_friendly": false,
  "affected_services_basic": [
    "3"
  ],
  "month": 4,
  "year_month": "2020-04",
  "day_of_week": 1
}


In [290]:
with open('../../data/processed-data/nyct-delay-events.json', 'w') as f:
    json.dump(nyct_events_json, f)

## Export in CSV format

CSV format is more convenient for loading as a Pandas dataframe; we just need to turn the lists back into strings.

In [289]:
nyct_events_csv_export = nyct_events_renamed.copy()
nyct_events_csv_export["alert_descriptions"] = nyct_events_csv_export["alert_descriptions"].apply(lambda x: " | ".join(x))
nyct_events_csv_export["affected_services"] = nyct_events_csv_export["affected_services"].apply(lambda x: " | ".join(x))
nyct_events_csv_export["affected_services_basic"] = nyct_events_csv_export["affected_services_basic"].apply(lambda x: " | ".join(x))
nyct_events_csv_export["station_daytime_routes"] = nyct_events_csv_export["station_daytime_routes"].apply(lambda x: " | ".join(x))

In [291]:
nyct_events_csv_export.to_csv('../../data/processed-data/nyct-delay-events.csv', index=False)